From ekidata.jp, 4 csv files containing data about train stations in Japan were obtained: station.csv, line.csv, company.csv and join.csv.

Postcode data in Japan was obtained from post.japanpost.jp/zipcode/dl/utf-zip.html and is used to geocode the train stations.

Opening the csv files and reading the data into pandas dataframes, we can see that some of the data is written in Japanese, such as the columns 'station_name' and 'address' in station.csv.  We need to convert this data to English for easier analysis.

In [ ]:
import pandas as pd

# Load the csv files
station = pd.read_csv('station.csv')
line =  pd.read_csv('line.csv')
company = pd.read_csv('company.csv')
join =  pd.read_csv('join.csv')
postcode = pd.read_csv('postcode_jp.csv')

# Print the output 
station

Using cutlet.py module,  we can convert the Japanese text to English.  We will use the following code to convert all the data that are Japanese to English from each csv file, and put the translated columns to the right of the original column.

In [19]:
import pandas as pd
import cutlet as ct

# Load the csv files
station = pd.read_csv('station.csv')
line =  pd.read_csv('line.csv')
company = pd.read_csv('company.csv')
join =  pd.read_csv('join.csv')
postcode = pd.read_csv('postcode_jp.csv')

# Create a Cutlet object
cutlet_obj = ct.Cutlet()

# Define a function to translate Japanese text to English
def translate_jp_to_en(text):
    return cutlet_obj.romaji(text)

# Apply the translation function to the japanese-lettered columns
station['station_name_en'] = station['station_name'].apply(translate_jp_to_en)
station['address_en'] = station['address'].apply(translate_jp_to_en)

line['line_name_en'] = line['line_name'].apply(translate_jp_to_en)
line['line_name_h_en'] = line['line_name_h'].apply(translate_jp_to_en)

company['company_name_en'] = company['company_name'].apply(translate_jp_to_en)
company['company_name_h_en'] = company['company_name_h'].apply(translate_jp_to_en)

postcode['prefecture_en'] = postcode['prefecture'].apply(translate_jp_to_en)
postcode['city_en'] = postcode['city'].apply(translate_jp_to_en)

# Get the current column order
cols_s = station.columns.tolist()
cols_l = line.columns.tolist()
cols_c = company.columns.tolist()
cols_p = postcode.columns.tolist()

# Move english-lettered columns to the right of japanese-lettered columns
cols_s.insert(cols_s.index('station_name') + 1, cols_s.pop(cols_s.index('station_name_en')))
cols_s.insert(cols_s.index('address') + 1, cols_s.pop(cols_s.index('address_en')))

cols_l.insert(cols_l.index('line_name') + 1, cols_l.pop(cols_l.index('line_name_en')))
cols_l.insert(cols_l.index('line_name_h') + 1, cols_l.pop(cols_l.index('line_name_h_en')))

cols_c.insert(cols_c.index('company_name') + 1, cols_c.pop(cols_c.index('company_name_en')))
cols_c.insert(cols_c.index('company_name_h') + 1, cols_c.pop(cols_c.index('company_name_h_en')))

cols_p.insert(cols_p.index('prefecture') + 1, cols_p.pop(cols_p.index('prefecture_en')))
cols_p.insert(cols_p.index('city') + 1, cols_p.pop(cols_p.index('city_en')))

# Reorder the DataFrame columns
station = station[cols_s]
line = line[cols_l]
company = company[cols_c]
postcode = postcode[cols_p]


# Print the output
station # OR line, company, postcode

To make it easier for future data analysis, the translated 'city' and 'prefecture' columns from postcode.csv are merged into the station.csv file. 

This is done by inner join both the tables (left:station, right:postcode) based on the 'post' and 'postcode' column respectively. However, preliminary check on the 'post' data in the station.csv file shows that the value need cleaning like removing "-" while the 'postcode' data in 'postcode_csv' needs changing datatype to string. Some 7 digits string data in 'post' also has leading "0 (zero)" which need truncating using lambda function. 

The edited tables (station, line, company) are then outputted as a new csv file to be used in data analysis.

In [ ]:
# Remove '-' from the data in 'post' column
station['post'] = station['post'].str.replace('-', '')

# Convert the 'postcode' column to string type
postcode['postcode'] = postcode['postcode'].astype(str)

# Remove the leading '0' from the 'post' column if it is 7 digits
station['post'] = station['post'].apply(lambda x: x.lstrip('0') if len(x) > 6 else x)

# Merge station.csv and postcode_jp.csv based on 'post' and 'postcode' columns
station = pd.merge(station, postcode[['postcode', 'city_en', 'prefecture_en']], left_on='post', right_on='postcode', how='inner')

# Rename the 'prefecture_en' to 'prefecture'
station = station.rename(columns={'prefecture_en': 'prefecture'})


# Output to csv file
station.to_csv('station_updated.csv', index=False)
company.to_csv('company_edited.csv', index=False)
line.to_csv('line_edited.csv',  index=False)

print("All processed data has been saved to CSV files")